# Convert Heatmap -> GIF Timelapse

In [ ]:
import folium
import folium.plugins as plugins

import os
import fileinput

from GrabzIt import GrabzItImageOptions
from GrabzIt import GrabzItClient
import imageio
from pathlib import Path

In [22]:
# Extract cleaned data
data = list()

with open('heatmap_data.txt', 'r') as f:
    for line in f:
        numLine = int(line)
        temp = list()
        for i in range(numLine):
            newLine = [float(x) for x in f.readline().replace('\n', '').split(',')]
            temp.append(newLine)
        data.append(temp)

In [23]:
# Create year and season list
season_index = list()
season = ['Spring', 'Summer', 'Autumn', 'Winter']

for i in range(2000, 2023):
    for j in season:
        season_index.append(str(i) + ' - ' + j)

In [24]:
# convert heatmap timelapse to individual HTML file
path = "GIF_HTML"
isExist = os.path.exists(path)

if not isExist:
   os.makedirs(path)

seasonList = ['Spring (March to May)', 'Summer (June to August)', 'Autumn (September to November)', 'Winter (December to February)']
for i in range(2000, 2023):
    for j in range(4):
        m_i = folium.Map(location = [36.2048, 138.2529], zoom_start = 5)

        # Add year & season label/title to the map
        title_html = '''
                     <h3 align="center" style="font-size:22px">
                        <b>
                            {}
                        </b>
                    </h3>
                     '''.format('Season: ' + str(i) + ' - ' + seasonList[j])   
        m_i.get_root().html.add_child(folium.Element(title_html))

        # Plot colors to the map using json geographical borderst
        plugins.HeatMap(
            data[j + 4*(i - 2000)],
            radius = 25,
            blur = 15,
            gradient = {0 : 'navy', 0.6 : 'cyan', 0.7 : 'lime', 0.8 : 'yellow', 1 : 'red'}
        ).add_to(m_i)

        folium.TileLayer('stamenterrain').add_to(m_i)

        # save as html file
        m_i.save("{folder}/{year}_{index}.html".format(folder = path, year = i, index = j))

        with fileinput.FileInput("{folder}/{year}_{index}.html".format(folder = path, year = i, index = j), inplace=True) as file:
             for line in file:
                # set zoomControl to false to prevent zoom in/out during conversion to image
                print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [16]:
# convert HTML to jpg using GrabzIt API, this might take several minutes
path_img = "GIF_Img"
path_html = "GIF_HTML"
isExist = os.path.exists(path_img)

if not isExist:
   os.makedirs(path_img)

# use dummy account key
grabzIt = GrabzItClient.GrabzItClient("MTQ5YzJhNjg2M2IyNDg3ZmE5ZDVjMmQ3ZDUxMTY2MmE", "VVN_Ifl2UfNnqXisBmi5T7sjXAEcRLteBFMB3KOvRqM")

options = GrabzItImageOptions.GrabzItImageOptions()
options.format = "jpg"

for i in range(2000, 2023):
    for j in range(4):
        grabzIt.FileToImage("{folder}/{year}_{index}.html".format(folder = path_html, year = i, index = j), options)
        grabzIt.SaveTo("{folder}/{year}_{index}.jpg".format(folder = path_img, year = i, index = j))

In [18]:
# convert image files to 1 GIF
image_path = Path('GIF_Img')

images = list(image_path.glob('*.jpg'))
image_list = []
for file_name in images:
    image_list.append(imageio.v2.imread(file_name))
    
imageio.mimwrite('Heatmap_Timelapse.gif', image_list, fps=1)

In [19]:
# OPTIONAL
# run this cell to remove all the HTML file and folder

path = 'GIF_HTML'
for i in range(2000, 2023):
    for j in range(4):
        os.remove("{folder}/{year}_{index}.html".format(folder = path, year = i, index = j))

os.rmdir(path)

In [ ]:
# OPTIONAL
# run this cell to remove all the jpg file and folder

path = 'GIF_Img'
for i in range(2000, 2023):
    for j in range(4):
        os.remove("{folder}/{year}_{index}.jpg".format(folder = path, year = i, index = j))

os.rmdir(path)